### 固有値問題

In [2]:
:dep nalgebra

In [3]:
extern crate nalgebra as na;

use na::{DMatrix, SymmetricEigen};

let a = DMatrix::from_row_slice(3,3, &[
    1.0, 0.5, 0.5,
    0.5, 1.0, 0.5,
    0.5, 0.5, 1.0,
]);

let eigen = SymmetricEigen::new(a);

println!("Eigenvalues: {:?}", eigen.eigenvalues);
println!("Eigenvectors: {:?}", eigen.eigenvectors);

Eigenvalues: VecStorage { data: [2.000000000000001, 0.5000000000000002, 0.5], nrows: Dyn(3), ncols: Const }
Eigenvectors: VecStorage { data: [0.5773502691896257, 0.5773502691896257, 0.5773502691896258, -0.816496580927726, 0.40824829046386296, 0.4082482904638631, 0.0, 0.7071067811865477, -0.7071067811865475], nrows: Dyn(3), ncols: Dyn(3) }


### 勾配法